# Using timecorr function

Blah blah

# Load in the required libraries

In [ ]:
import timecorr as tc
import numpy as np

# Explain weighted dynamic correlation calculation

In [ ]:
#wcorred_data = tc.wcorr(np.array(subs_data_1),  np.array(subs_data_2), weights=laplace['weights'](T))